# Prospect + specutils + SPARCL

*Credit*: [Eric Armengaud](https://github.com/armengau), Saclay - CEA, is the primary author of prospect.  See also the [prospect contributors](https://github.com/desihub/prospect/graphs/contributors).

*Project*: Obtain DESI EDR spectra using the [NOIRLab SPARCL spectrum service](https://astrosparcl.datalab.noirlab.edu), convert data to [specutils objects](https://specutils.readthedocs.io/en/stable/) as needed, and use [prospect](https://desi-prospect.readthedocs.io/en/latest/) to display the data.

*Takeaway*: At no point in this notebook will any data files be opened.

*Takeaway*: prospect allows multiple, independent spectra visualizations to coexist within the same notebook.

## Imports

Note that we are not relying heavily on the DESI software stack, although some imports will be embedded within `prospect`.

In [ ]:
import numpy as np
import astropy.units as u
from astropy.nddata import InverseVariance
from specutils import __version__ as specutils_version, Spectrum1D
from prospect import __version__ as prospect_version
from prospect.viewer import plotspectra
from prospect.specutils import Spectra
from sparcl import __version__ as sparcl_version
from sparcl.client import SparclClient
from dl import version as dl_version, queryClient as qc
print(f"astro-datalab=={dl_version}")
print(f"specutils=={specutils_version}")
print(f"prospect=={prospect_version}")
print(f"sparcl=={sparcl_version}")

## Start SPARCL Client

In [ ]:
client = SparclClient()
client

## Set up Data Lab database interface

`db01` can be set up for testing, but should not currently be necessary.

In [ ]:
# qc.set_profile('db01')

## DESI

### Find DESI spectra

SPARCL provides access to DESI spectra that have been coadded by HEALPixel.  This corresponds to entries in the `desi_edr.zpix` table.

In [ ]:
q = """SELECT z.targetid, z.chi2, z.z, z.zerr, z.zwarn, z.spectype, z.subtype, z.coadd_numexp, z.coadd_exptime, z.healpix, z.deltachi2
FROM desi_edr.zpix AS z
WHERE z.zcat_primary AND z.survey = 'sv3' AND z.program = 'dark' AND z.spectype = 'GALAXY' AND z.z BETWEEN 0.5 AND 0.9 AND z.zwarn = 0 ORDER BY z.targetid LIMIT 50;
"""
desi_ids = qc.query(sql=q, fmt='table')
desi_ids

**QA**: Do we really have 50 unique spectra?

In [ ]:
assert (np.unique(desi_ids['targetid']) == desi_ids['targetid']).all()

### Retrieve DESI spectra

With the set of `targetid` obtained above, we can directly retrieve DESI spectra.

In [ ]:
include = client.get_all_fields(dataset_list=['DESI-EDR'])
desi_spectra = client.retrieve_by_specid(desi_ids['targetid'].value.tolist(), include=include, dataset_list=['DESI-EDR'])
desi_spectra.info

**QA**: Did we really find all of the expected spectra?

In [ ]:
assert (np.array([r.targetid for r in sorted(desi_spectra.records, key=lambda x: x.targetid)]) == desi_ids['targetid']).all()
assert all([r.survey == 'sv3' for r in desi_spectra.records])
assert all([r.program == 'dark' for r in desi_spectra.records])

**QA**: Do all spectra records have the same wavelength solution?

In [ ]:
assert all([(r.wavelength == desi_spectra.records[0].wavelength).all() for r in desi_spectra.records])

In [ ]:
desi_spectra.records[0]

### Organize metadata

Prospect needs several inputs:

1. An object containing spectra.  In this case we'll use [`prospect.specutils.Spectra`](https://desi-prospect.readthedocs.io/en/latest/api.html#prospect.specutils.Spectra), which inherits from [`SpectrumList`](https://specutils.readthedocs.io/en/stable/api/specutils.SpectrumList.html#specutils.SpectrumList), and is really just a [`Spectrum1D`](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D) object underneath.
   * The object contains the usual flux, wavelength, uncertainty.
   * In addtion a "fibermap" table is needed. This should be an Astropy `Table` with the expected columns.
2. A redshift catalog. This should be an Astropy `Table` with the expected columns.
3. A model spectrum.  The model is actually provided by SPARCL, but we need to input it separately.

#### Spectrum object

First we assemble the components of the spectrum object

In [ ]:
flux = np.zeros((len(desi_spectra.records), desi_spectra.records[0].flux.shape[0]), dtype=desi_spectra.records[0].flux.dtype)
uncertainty = np.zeros((len(desi_spectra.records), desi_spectra.records[0].ivar.shape[0]), dtype=desi_spectra.records[0].ivar.dtype)
mask = np.zeros((len(desi_spectra.records), desi_spectra.records[0].mask.shape[0]), dtype=desi_spectra.records[0].mask.dtype)
meta = {'sparcl_id': list(), 'data_release': list()}
sparcl_id = list()
data_release = list()
for k in range(len(desi_spectra.records)):
    flux[k, :] = desi_spectra.records[k].flux
    uncertainty[k, :] = desi_spectra.records[k].ivar
    mask[k, :] = desi_spectra.records[k].mask
    meta['sparcl_id'].append(desi_spectra.records[k].sparcl_id)
    meta['data_release'].append(desi_spectra.records[k].data_release)

And the "fibermap" table. We'll start with photometric quantities.

In [ ]:
columns = ('targetid', 'ra', 'dec', 'ref_epoch', 'pmra', 'pmdec', 'ebv', 'flux_g', 'flux_r', 'flux_z', 'flux_w1', 'flux_w2')
q = """SELECT {0}
FROM desi_edr.photometry
WHERE targetid IN ({1}) ORDER BY targetid;
""".format(', '.join(columns), ', '.join([str(t) for t in desi_ids['targetid'].value.tolist()]))
fibermap = qc.query(sql=q, fmt='table')
for col in fibermap.colnames:
    if col == 'ra' or col == 'dec':
        fibermap.rename_column(col, 'TARGET_' + col.upper())
    else:
        fibermap.rename_column(col, col.upper())

**QA**: Did we find photometry for every `targetid`?

In [ ]:
assert (fibermap['TARGETID'] == desi_ids['targetid']).all()

Next we add targeting bitmasks.

In [ ]:
columns = ('targetid', 'sv3_desi_target', 'sv3_bgs_target', 'sv3_mws_target', 'sv3_scnd_target')
q = """SELECT DISTINCT {0}
FROM desi_edr.target
WHERE targetid IN ({1}) ORDER BY targetid;
""".format(', '.join(columns), ', '.join([str(t) for t in fibermap['TARGETID'].value.tolist()]))
targeting = qc.query(sql=q, fmt='table')
for col in targeting.colnames:
    targeting.rename_column(col, col.upper())

**QA**: Did we find targeting for every `targetid`?

In [ ]:
assert (targeting['TARGETID'] == fibermap['TARGETID']).all()

Add columns into `fibermap`.

In [ ]:
for col in ('SV3_DESI_TARGET', 'SV3_BGS_TARGET', 'SV3_MWS_TARGET', 'SV3_SCND_TARGET'):
    fibermap.add_column(targeting[col])
fibermap

Finally assemble the object.

In [ ]:
desi_prospect = Spectra(bands=['coadd'],
                        wave={'coadd': desi_spectra.records[0].wavelength},
                        flux={'coadd': flux},
                        ivar={'coadd': uncertainty},
                        mask={'coadd': mask},
                        fibermap=fibermap,
                        meta={'coadd': meta})

#### Redshift catalog

We can re-use the initial query above; it was deliberately constructed.

In [ ]:
desi_zcatalog = desi_ids.copy()
for col in desi_zcatalog.colnames:
    if col == 'healpix':
        desi_zcatalog.rename_column(col, 'HPXPIXEL')
    else:
        desi_zcatalog.rename_column(col, col.upper())
desi_zcatalog

#### Model spectra

Prospect expects a model in the form of a tuple containing wavelength and flux. Since SPARCL provides the model, this is easy.  There are other ways to specify the model, but these require more access to the DESI software stack *and* data *files*.

In [ ]:
model_flux = np.zeros((len(desi_spectra.records), desi_spectra.records[0].model.shape[0]), dtype=desi_spectra.records[0].model.dtype)
for k in range(len(desi_spectra.records)):
        model_flux[k, :] = desi_spectra.records[k].model
desi_model = (desi_spectra.records[0].wavelength, model_flux)

### Start prospect

With everything assembled, the interface to prospect is just a single call.

In [ ]:
plotspectra(desi_prospect, zcatalog=desi_zcatalog, redrock_cat=None, notebook=True, with_thumb_tab=False, with_vi_widgets=False, with_coaddcam=False, mask_type='SV3_DESI_TARGET',
            model_from_zcat=False, model=desi_model)

## SDSS

### Find SDSS spectra

In [ ]:
q = """SELECT z.specobjid, z.bestobjid, z.z, z.zerr, z.zwarning, z.class, z.subclass, z.rchi2diff, z.primtarget, z.sectarget
FROM sdss_dr16.specobj AS z
WHERE z.bestobjid > 0 AND z.run2d = '26' AND z.plate = 2955 AND z.mjd = 54562 AND z.class = 'GALAXY' AND z.zwarning = 0 ORDER BY z.specobjid LIMIT 50;
"""
sdss_ids = qc.query(sql=q, fmt='table')
sdss_ids

**QA**: Do we really have 50 unique spectra?

In [ ]:
assert (np.unique(sdss_ids['specobjid']) == sdss_ids['specobjid']).all()

### Retrieve SDSS spectra

With the set of `specobjid` obtained above, we can directly retrieve SDSS spectra.

In [ ]:
include = client.get_all_fields(dataset_list=['SDSS-DR16'])
sdss_spectra = client.retrieve_by_specid(sdss_ids['specobjid'].value.tolist(), include=include, dataset_list=['SDSS-DR16'])
sdss_spectra.info

**QA**: Did we really find all of the expected spectra? Note that we have to assume `specid == specobjid`.

In [ ]:
assert (np.array([r.specid for r in sorted(sdss_spectra.records, key=lambda x: x.specid)]) == sdss_ids['specobjid']).all()
assert all([r.plate == 2955 for r in sdss_spectra.records])
assert all([r.mjd == 54562 for r in sdss_spectra.records])

**QA**: Do all spectra records have the same wavelength solution?

In [ ]:
assert all([(r.wavelength == sdss_spectra.records[0].wavelength).all() for r in sdss_spectra.records])

In [ ]:
sdss_spectra.records[0]

### Organize metadata

Prospect needs several inputs:

1. An object containing spectra.  In this case we'll use a [`Spectrum1D`](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D) object.
   * The object contains the usual flux, wavelength, uncertainty.
   * In addtion a "plugmap" table is needed. This should be an Astropy `Table` with the expected columns.
2. A redshift catalog. This should be an Astropy `Table` with the expected columns.
3. A model spectrum.  The model is actually provided by SPARCL, but we need to input it separately.

#### Spectrum object

First we assemble the components of the spectrum object

In [ ]:
flux = np.zeros((len(sdss_spectra.records), sdss_spectra.records[0].flux.shape[0]), dtype=sdss_spectra.records[0].flux.dtype)
uncertainty = np.zeros((len(sdss_spectra.records), sdss_spectra.records[0].ivar.shape[0]), dtype=sdss_spectra.records[0].ivar.dtype)
mask = np.zeros((len(sdss_spectra.records), sdss_spectra.records[0].mask.shape[0]), dtype=sdss_spectra.records[0].mask.dtype)
meta = {'sparcl_id': list(), 'data_release': list()}
sparcl_id = list()
data_release = list()
for k in range(len(sdss_spectra.records)):
    flux[k, :] = sdss_spectra.records[k].flux
    uncertainty[k, :] = sdss_spectra.records[k].ivar
    mask[k, :] = sdss_spectra.records[k].mask
    meta['sparcl_id'].append(sdss_spectra.records[k].sparcl_id)
    meta['data_release'].append(sdss_spectra.records[k].data_release)

And the "plugmap" table. We'll start with photometric quantities.

In [ ]:
columns = ('objid', 'ra', 'dec', 'u', 'g', 'r', 'i', 'z')
q = """SELECT {0}
FROM sdss_dr16.photoplate
WHERE objid IN ({1}) ORDER BY objid;
""".format(', '.join(columns), ', '.join([str(t) for t in sdss_ids['bestobjid'].value.tolist()]))
plugmap = qc.query(sql=q, fmt='table')
for col in plugmap.colnames:
    plugmap.rename_column(col, col.upper())
mag = np.zeros((len(plugmap), 5), dtype=plugmap['G'].value.dtype)
for k, band in enumerate('UGRIZ'):
    mag[:, k] = plugmap[band].value
plugmap.add_column(mag, name='MAG')
plugmap.add_column(sdss_ids['primtarget'], name='PRIMTARGET')
plugmap.add_column(sdss_ids['sectarget'], name='SECTARGET')
meta['plugmap'] = plugmap

**QA**: Did we find photometry for every `bestobjid`?

In [ ]:
assert (plugmap['OBJID'] == sorted(sdss_ids['bestobjid'])).all()

Finally assemble the object.

In [ ]:
sdss_prospect = Spectrum1D(flux=flux * u.Unit('1e-17 erg / (Angstrom cm2 s)'),
                           spectral_axis=sdss_spectra.records[0].wavelength * u.Unit('Angstrom'),
                           uncertainty=InverseVariance(uncertainty),
                           mask=mask != 0,
                           meta=meta)

#### Redshift catalog

We can re-use the initial query above; it was deliberately constructed.

In [ ]:
sdss_zcatalog = sdss_ids.copy()
for col in sdss_zcatalog.colnames:
    if col == 'zerr':
        sdss_zcatalog.rename_column(col, 'Z_ERR')
    else:
        sdss_zcatalog.rename_column(col, col.upper())
sdss_zcatalog

#### Model spectra

Prospect expects a model in the form of a tuple containing wavelength and flux. Since SPARCL provides the model, this is easy.  There are other ways to specify the model, but these require more access to the SDSS *files*.

In [ ]:
model_flux = np.zeros((len(sdss_spectra.records), sdss_spectra.records[0].model.shape[0]), dtype=sdss_spectra.records[0].model.dtype)
for k in range(len(sdss_spectra.records)):
        model_flux[k, :] = sdss_spectra.records[k].model
sdss_model = (sdss_spectra.records[0].wavelength, model_flux)

### Start prospect

With everything assembled, the interface to prospect is just a single call.

In [ ]:
plotspectra(sdss_prospect, zcatalog=sdss_zcatalog, redrock_cat=None, notebook=True, with_thumb_tab=False, with_vi_widgets=False, with_coaddcam=False, mask_type='PRIMTARGET',
            model_from_zcat=False, model=sdss_model)

## (e)BOSS Spectra

In [ ]:
q = """SELECT z.specobjid, z.bestobjid, z.z, z.zerr, z.zwarning, z.class, z.subclass, z.rchi2diff, z.boss_target1, z.eboss_target0, z.eboss_target1, z.eboss_target2
FROM sdss_dr16.specobj AS z
WHERE z.bestobjid > 0 AND z.run2d = 'v5_13_0' AND z.plate = 9599 AND z.mjd = 58131 AND z.class = 'GALAXY' AND z.zwarning = 0 ORDER BY z.specobjid LIMIT 50;
"""
boss_ids = qc.query(sql=q, fmt='table')
boss_ids

**QA**: Do we really have 50 unique spectra?

In [ ]:
assert (np.unique(boss_ids['specobjid']) == boss_ids['specobjid']).all()

### Retrieve (e)BOSS spectra

With the set of `specobjid` obtained above, we can directly retrieve (e)BOSS spectra.

In [ ]:
include = client.get_all_fields(dataset_list=['BOSS-DR16'])
boss_spectra = client.retrieve_by_specid(boss_ids['specobjid'].value.tolist(), include=include, dataset_list=['BOSS-DR16'])
boss_spectra.info

**QA**: Did we really find all of the expected spectra?

In [ ]:
assert (np.array([r.specid for r in sorted(boss_spectra.records, key=lambda x: x.specid)]) == boss_ids['specobjid']).all()
assert all([r.plate == 9599 for r in boss_spectra.records])
assert all([r.mjd == 58131 for r in boss_spectra.records])

**QA**: Do all spectra records have the same wavelength solution?

In [ ]:
assert all([(r.wavelength == boss_spectra.records[0].wavelength).all() for r in boss_spectra.records])

In [ ]:
boss_spectra.records[0]

### Organize metadata

Prospect needs several inputs:

1. An object containing spectra.  In this case we'll use a [`Spectrum1D`](https://specutils.readthedocs.io/en/stable/api/specutils.Spectrum1D.html#specutils.Spectrum1D) object.
   * The object contains the usual flux, wavelength, uncertainty.
   * In addtion a "plugmap" table is needed. This should be an Astropy `Table` with the expected columns.
2. A redshift catalog. This should be an Astropy `Table` with the expected columns.
3. A model spectrum.  The model is actually provided by SPARCL, but we need to input it separately.

#### Spectrum object

First we assemble the components of the spectrum object

In [ ]:
flux = np.zeros((len(boss_spectra.records), boss_spectra.records[0].flux.shape[0]), dtype=boss_spectra.records[0].flux.dtype)
uncertainty = np.zeros((len(boss_spectra.records), boss_spectra.records[0].ivar.shape[0]), dtype=boss_spectra.records[0].ivar.dtype)
mask = np.zeros((len(boss_spectra.records), boss_spectra.records[0].mask.shape[0]), dtype=boss_spectra.records[0].mask.dtype)
meta = {'sparcl_id': list(), 'data_release': list()}
sparcl_id = list()
data_release = list()
for k in range(len(boss_spectra.records)):
    flux[k, :] = boss_spectra.records[k].flux
    uncertainty[k, :] = boss_spectra.records[k].ivar
    mask[k, :] = boss_spectra.records[k].mask
    meta['sparcl_id'].append(boss_spectra.records[k].sparcl_id)
    meta['data_release'].append(boss_spectra.records[k].data_release)

And the "plugmap" table. We'll start with photometric quantities.

In [ ]:
columns = ('objid', 'ra', 'dec', 'u', 'g', 'r', 'i', 'z')
q = """SELECT {0}
FROM sdss_dr16.photoplate
WHERE objid IN ({1}) ORDER BY objid;
""".format(', '.join(columns), ', '.join([str(t) for t in boss_ids['bestobjid'].value.tolist()]))
plugmap = qc.query(sql=q, fmt='table')
for col in plugmap.colnames:
    plugmap.rename_column(col, col.upper())
mag = np.zeros((len(plugmap), 5), dtype=plugmap['G'].value.dtype)
for k, band in enumerate('UGRIZ'):
    mag[:, k] = plugmap[band].value
plugmap.add_column(mag, name='MAG')
plugmap.add_column(boss_ids['boss_target1'], name='BOSS_TARGET1')
plugmap.add_column(boss_ids['eboss_target0'], name='EBOSS_TARGET0')
plugmap.add_column(boss_ids['eboss_target1'], name='EBOSS_TARGET1')
plugmap.add_column(boss_ids['eboss_target2'], name='EBOSS_TARGET2')
meta['plugmap'] = plugmap

**QA**: Did we find photometry for every `bestobjid`?

In [ ]:
assert (plugmap['OBJID'] == sorted(boss_ids['bestobjid'])).all()

Finally assemble the object.

In [ ]:
boss_prospect = Spectrum1D(flux=flux * u.Unit('1e-17 erg / (Angstrom cm2 s)'),
                           spectral_axis=boss_spectra.records[0].wavelength * u.Unit('Angstrom'),
                           uncertainty=InverseVariance(uncertainty),
                           mask=mask != 0,
                           meta=meta)

#### Redshift catalog

We can re-use the initial query above; it was deliberately constructed.

In [ ]:
boss_zcatalog = boss_ids.copy()
for col in boss_zcatalog.colnames:
    if col == 'zerr':
        boss_zcatalog.rename_column(col, 'Z_ERR')
    else:
        boss_zcatalog.rename_column(col, col.upper())
boss_zcatalog

#### Model spectra

Prospect expects a model in the form of a tuple containing wavelength and flux. Since SPARCL provides the model, this is easy.  There are other ways to specify the model, but these require more access to the (e)BOSS *files*.

In [ ]:
model_flux = np.zeros((len(boss_spectra.records), boss_spectra.records[0].model.shape[0]), dtype=boss_spectra.records[0].model.dtype)
for k in range(len(boss_spectra.records)):
        model_flux[k, :] = boss_spectra.records[k].model
boss_model = (boss_spectra.records[0].wavelength, model_flux)

### Start prospect

With everything assembled, the interface to prospect is just a single call.

In [ ]:
plotspectra(boss_prospect, zcatalog=boss_zcatalog, redrock_cat=None, notebook=True, with_thumb_tab=False, with_vi_widgets=False, with_coaddcam=False, mask_type='EBOSS_TARGET1',
            model_from_zcat=False, model=boss_model)